## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from observation import Sdataset, Analysis
import pandas as pd

def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

def add_nbre_pdc(data, name='ok', pr=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    #data[name] = data[name] & data['nbre_pdc_ok']
    if pr:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04']
date= dates[-1]
log = {'date_irve': date, 
       'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
      'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
#      'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  46565


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 1 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  791
nombre de pdc en itinerance   :  45774


In [6]:
# arborescence des champs et nombre de valeurs différentes
il = Sdataset(itinerance_init)
print(il.tree())

-1: root-derived (45774)
   0 : index (45774)
      1 : contact_operateur (141)
      2 : nom_enseigne (1285)
      3 : coordonneesXY (14491)
      4 : adresse_station (13738)
      5 : id_station_itinerance (16770)
      6 : nom_station (13458)
      7 : implantation_station (5)
      8 : nbre_pdc (50)
      9 : condition_acces (2)
      10: horaires (420)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (37092)
      13: date_maj (548)
      14: last_modified (511)


-----------------------
## 4 - Bilan initial intégrité
- plus de 50 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                          14993
contact_operateur - id_station_itinerance          12370
nom_enseigne - id_station_itinerance                7903
coordonneesXY - id_station_itinerance              13436
id_station_itinerance - id_pdc_itinerance           1120
nom_station - id_station_itinerance                 2647
implantation_station - id_station_itinerance         290
nbre_pdc - id_station_itinerance                    1808
condition_acces - id_station_itinerance               23
horaires - id_station_itinerance                   12132
station_deux_roues - id_station_itinerance         13191
adresse_station - coordonneesXY                     1762

nombre d'enregistrements sans erreurs :  27507
nombre d'enregistrements avec au moins une erreur :  18267
taux d'erreur :  40  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- plus de la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de plus de 80% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  8682  soit  48  %
nombre de pdc sans doublon   :  9585


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance            278
nom_enseigne - id_station_itinerance                 449
coordonneesXY - id_station_itinerance               1063
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1674
implantation_station - id_station_itinerance         210
nbre_pdc - id_station_itinerance                     581
condition_acces - id_station_itinerance               13
horaires - id_station_itinerance                     267
station_deux_roues - id_station_itinerance           719
adresse_station - coordonneesXY                      846

nombre d'enregistrements sans erreurs :  6407
nombre d'enregistrements avec au moins une erreur :  3178


-----------------------
## 6 - Séparation doublons station - date de maj
- plus de 15% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de plus de 20% le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 33 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['etape3_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  185  soit  6  %
nombre de pdc sans doublon   :  2993


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             13
nom_enseigne - id_station_itinerance                 265
coordonneesXY - id_station_itinerance                775
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                 1420
implantation_station - id_station_itinerance          49
nbre_pdc - id_station_itinerance                     484
condition_acces - id_station_itinerance                8
horaires - id_station_itinerance                      52
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                      742

nombre d'enregistrements sans erreurs :  155
nombre d'enregistrements avec au moins une erreur :  2838  soit  89  %


In [13]:
# structure des données résiduelles et nombre de valeurs
il = Sdataset(itinerance_5)
print(il.tree())

-1: root-derived (2838)
   0 : index (2838)
      1 : contact_operateur (38)
      2 : nom_enseigne (340)
      3 : coordonneesXY (1003)
      4 : adresse_station (1038)
      5 : id_station_itinerance (1010)
         13: date_maj (66)
      6 : nom_station (1587)
      7 : implantation_station (5)
      8 : nbre_pdc (17)
      9 : condition_acces (2)
      10: horaires (69)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (2838)
      14: last_modified (70)


In [14]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  93


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [15]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  2838
total des doublons à supprimer :  8867
nombre de pdc avec controles ok :  34069


In [16]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [17]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [18]:
# structure des données bonnes
il = Sdataset(itinerance_ok.loc[:, mandatory])
print(il.tree())

-1: root-derived (34069)
   11: id_pdc_itinerance (34069)
      4 : id_station_itinerance (15561)
         1 : nom_enseigne (965)
         2 : coordonneesXY (11340)
            0 : contact_operateur (127)
            3 : adresse_station (10714)
            8 : condition_acces (2)
         5 : nom_station (11406)
         6 : implantation_station (5)
         7 : nbre_pdc (50)
         9 : horaires (356)
         10: station_deux_roues (5)
         19: paiement_autre (7)
      12: puissance_nominale (74)
      13: prise_type_ef (8)
      14: prise_type_2 (8)
      15: prise_type_combo_ccs (7)
      16: prise_type_chademo (7)
      17: prise_type_autre (7)
      18: paiement_acte (8)
      20: reservation (6)
      21: accessibilite_pmr (4)
      22: restriction_gabarit (113)
      23: date_maj (510)
      24: last_modified (439)


### Indicateurs

In [19]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [20]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  79  % des écarts

support@virta.global 1031
supervision-ev.france@totalenergies.com 506
contact@e55c.com 269
pascal.lhermitte@siege27.fr 260
support@alizecharge.fr 176


In [21]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, pr=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko            826
29  %


In [22]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 29 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [23]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1809,43102,support@virta.global,MODULO,"[4.34044107,49.7691777]",Grande Rue,FRS08P08370,ROUVROY SUR AUBRY - 57688 - Grande Rue,Voirie,1,Accès libre,24/7,False,FRS08E57688,2023-05-31,2023-07-04T06:46:52.168000+00:00
1810,43103,support@virta.global,MODULO,"[4.62525700,49.8281160]",Rue Gambetta,FRS08P08490,VOUZIERS - 57629 - rue Gambetta,Voirie,2,Accès libre,24/7,False,FRS08E57629,2023-05-31,2023-07-04T06:46:52.168000+00:00
1811,43104,support@virta.global,MODULO,"[4.70136000,49.4002470]",Place Aristide Briand,FRS08P08419,SIGNY L’ABBAYE - Place Aristide Briand - 57593,Voirie,1,Accès libre,24/7,False,FRS08E57593,2023-05-31,2023-07-04T06:46:52.168000+00:00
1812,43105,support@virta.global,MODULO,"[4.49431027,49.7880936]",Place de l'Eglise,FRS08P08420,SIGNY LE PETIT - 57627 - Place de l'église,Voirie,2,Accès libre,24/7,False,FRS08E57627,2023-05-31,2023-07-04T06:46:52.168000+00:00
1813,43106,support@virta.global,MODULO,"[4.62525700,49.8281160]",Place de l'Eglise,FRS08P08420,SIGNY LE PETIT - 57628 - Place de l'église,Voirie,1,Accès libre,24/7,False,FRS08E57628,2023-05-31,2023-07-04T06:46:52.168000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,46466,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.58115, 43.280687]",PRAIRIE DE LA MER GRIMAUD,FR55CP83310,PRAIRIE DE LA MER GRIMAUD,Voirie,4,Accès réservé,24/7,FALSE,FR55CEFR833101910L1TT0RAL3,2022-11-03,2023-07-04T06:45:00.966000+00:00
2933,46467,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.581173, 43.276785]",PRAIRIE DE LA MER ENTREE GRIMAUD,FR55CP83310,PRAIRIE DE LA MER ENTREE GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR83310335AVDELAMER0,2022-11-03,2023-07-04T06:45:00.966000+00:00
2934,46468,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.581173, 43.276785]",PRAIRIE DE LA MER ENTREE GRIMAUD,FR55CP83310,PRAIRIE DE LA MER ENTREE GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR83310335AVDELAMER1,2022-11-03,2023-07-04T06:45:00.966000+00:00
2935,46469,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.581173, 43.276785]",PRAIRIE DE LA MER ENTREE GRIMAUD,FR55CP83310,PRAIRIE DE LA MER ENTREE GRIMAUD,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR83310335AVDELAMER2,2022-11-03,2023-07-04T06:45:00.966000+00:00


### Cohérence implantation_station - id_station
- 64 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [24]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1112,39201,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE205868TE,2022-07-25,2023-07-04T06:48:59.787000+00:00
1115,39312,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE200977,2022-07-25,2023-07-04T06:48:59.787000+00:00
1116,39328,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE201426,2022-07-25,2023-07-04T06:48:59.787000+00:00
1117,39405,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE200921,2022-07-25,2023-07-04T06:48:59.787000+00:00
1118,39407,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE201290,2022-07-25,2023-07-04T06:48:59.787000+00:00
1119,39439,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking public,6,Accès libre,24/7,0,FRQOVE205868,2022-07-25,2023-07-04T06:48:59.787000+00:00
1123,39588,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205871TE,2022-07-25,2023-07-04T06:48:58.042000+00:00
1124,39599,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE201396,2022-07-25,2023-07-04T06:48:58.042000+00:00
1125,39600,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205675,2022-07-25,2023-07-04T06:48:58.042000+00:00
1126,39601,contact@qovoltis.com,QOVOLTIS,"[-2.76491, 48.49415]",52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sain...,FRQOVP2200002,52 RUE DU GUÉ LAMBERT Hôtel Kyriad Direct Sai,Parking privé à usage public,6,Accès libre,24/7,0,FRQOVE205871,2022-07-25,2023-07-04T06:48:58.042000+00:00


### Cohérence nom_station - id_station
- 328 pdc sont associés à une station avec un nom non cohérent

In [25]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
71,8277,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM012,2023-06-02,2023-07-04T06:51:38.345000+00:00
72,8281,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM021,2023-06-02,2023-07-04T06:51:38.345000+00:00
73,8282,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM020,2023-06-02,2023-07-04T06:51:38.345000+00:00
74,8283,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM019,2023-06-02,2023-07-04T06:51:38.345000+00:00
75,8284,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM018,2023-06-02,2023-07-04T06:51:38.345000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988,46535,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S0,2022-11-03,2023-07-04T06:45:00.966000+00:00
2989,46536,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.90471, 43.942662]",PLACE EMMANUEL VITRIA - MORIERE,FR55CP84310,PLACE EMMANUEL VITRIA - MORIERE,Voirie,2,Accès réservé,24/7,FALSE,FR55CE843104394266249047121,2022-11-03,2023-07-04T06:45:00.966000+00:00
2990,46549,contact@e55c.com,ELECTRIC 55 CHARGING,"[1.683721, 47.862233]",RENAULT DEALER - BAIN DE BRETAGNE,FR55CPRNO35,RENAULT DEALER - BAIN DE BRETAGNE,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR354706LANSERVA0,2022-11-03,2023-07-04T06:45:00.966000+00:00
2991,46550,contact@e55c.com,ELECTRIC 55 CHARGING,"[1.683721, 47.862233]",RENAULT DEALER - BAIN DE BRETAGNE,FR55CPRNO35,RENAULT DEALER - BAIN DE BRETAGNE,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR354706LANSERVA1,2022-11-03,2023-07-04T06:45:00.966000+00:00


### Cohérence adresse - coordonnées
- 440 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [26]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,1546,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA PLACE ETIENNE DOLET,FRSIGPSIGE34211,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34211,2023-07-05,2023-07-05T02:53:47.855000+00:00
1,1547,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA PLACE ETIENNE DOLET,FRSIGPSIGE34212,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34212,2023-07-05,2023-07-05T02:53:47.855000+00:00
2,1550,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA,FRSIGPSIGE34412,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34412,2023-07-05,2023-07-05T02:53:47.855000+00:00
3,1551,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA,FRSIGPSIGE34411,SIGEIF - 4 RUE CRISTINO GARCIA PLACE ETIENNE D...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34411,2023-07-05,2023-07-05T02:53:47.855000+00:00
4,2994,sav@izivia.com,SOREGIES,"[0.210555, 46.92054]",ALLÉE DES MARRONNIERS,FRS86PSOREB1531,MONTS SUR GUESNES MEMORIAL 1,Voirie,2,Accès libre,24/7,false,FRS86ESOREB1531,2023-07-05,2023-07-05T02:53:44.159000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,46454,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S1,2022-11-03,2023-07-04T06:45:00.966000+00:00
2936,46470,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP731,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S0,2022-11-03,2023-07-04T06:45:00.966000+00:00
2937,46471,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP731,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S1,2022-11-03,2023-07-04T06:45:00.966000+00:00
2938,46472,contact@e55c.com,ELECTRIC 55 CHARGING,"[5.976272, 45.526596]",BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,FR55CPBP731,BP - AIRE DE L'ABIS - SAINT-JEOIRE-PRIEURE - 7...,Voirie,3,Accès réservé,24/7,FALSE,FR55CEFR7319043AB1S2,2022-11-03,2023-07-04T06:45:00.966000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [27]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
100,8841,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-07-04T06:51:38.345000+00:00
101,8842,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-07-04T06:51:38.345000+00:00
102,8843,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-07-04T06:51:38.345000+00:00
103,8866,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-07-04T06:51:38.345000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [28]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1879,43217,support@virta.global,MODULO,"[4.70237100,49.9925440]",Rue de la Gare,FRS08P08365,RIMOGNE-Rue de la Gare - 57624,Voirie,1,Accès libre,24/7,False,FRS08E57624,2023-05-31,2023-07-04T06:46:52.168000+00:00
1880,43218,support@virta.global,MODULO,"[4.59987580,49.8733253]",Rue Noël Champagne,FRS08P08367,ROCROI - 57693 - Rue Noël Champagne,Voirie,2,Accès libre,24/7,False,FRS08E57693,2023-05-31,2023-07-04T06:46:52.168000+00:00
1881,43219,support@virta.global,MODULO,"[4.384208,48.95369]",Rue du collége,FRS08P08311,Mouzon 35484 - Rue du collège,Voirie,2,Accès libre,24/7,False,FRS08E35484,2023-05-31,2023-07-04T06:46:52.168000+00:00
1882,43220,support@virta.global,MODULO,"[0.74696900,47.3465010]",Place de la mairie,FRS08P08303,MONTHOIS - 57717 - Place de la Mairie,Voirie,1,Accès libre,24/7,False,FRS08E57717,2023-05-31,2023-07-04T06:46:52.168000+00:00
1883,43221,support@virta.global,MODULO,"[1.20841700,47.6820550]",Place de la mairie,FRS08P08303,MONTHOIS - 57716 - Place de la Mairie,Voirie,2,Accès libre,24/7,False,FRS08E57716,2023-05-31,2023-07-04T06:46:52.168000+00:00
1884,43222,support@virta.global,MODULO,"[4.84286510,49.8835415]",Place Jean Baptiste Clément,FRS08P08302,MONTHERMÉ - 57542 - Pl Jean Baptiste Clément,Voirie,1,Accès libre,24/7,False,FRS08E57542,2023-05-31,2023-07-04T06:46:52.168000+00:00
1885,43223,support@virta.global,MODULO,"[4.70237100,49.9925440]",Gare de Lucquy,FRS08P08262,LUCQUY - 57625 - Gare,Voirie,2,Accès libre,24/7,False,FRS08E57625,2023-05-31,2023-07-04T06:46:52.168000+00:00
1886,43224,support@virta.global,MODULO,"[4.49431027,49.7880936]",Gare de Lucquy,FRS08P08262,LUCQUY - 57626 - Gare,Voirie,1,Accès libre,24/7,False,FRS08E57626,2023-05-31,2023-07-04T06:46:52.168000+00:00
1887,43225,support@virta.global,MODULO,"[4.27918560,49.9031852]",Place de la Mairie,FRS08P08264,MACHAULT - 57608 - Place de la Mairie,Voirie,2,Accès libre,24/7,False,FRS08E57608,2023-05-31,2023-07-04T06:46:52.168000+00:00
1888,43226,support@virta.global,MODULO,"[4.69976130,49.3958749]",Place de la Mairie,FRS08P08264,MACHAULT - 57609 - Place de la Mairie,Voirie,1,Accès libre,24/7,False,FRS08E57609,2023-05-31,2023-07-04T06:46:52.168000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [29]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
578,13904,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-07-04T06:50:41.957000+00:00
579,13905,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-07-04T06:50:41.957000+00:00
580,13907,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-07-04T06:50:41.957000+00:00
581,13908,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-07-04T06:50:41.957000+00:00
582,13909,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-07-04T06:50:41.957000+00:00
583,13910,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-07-04T06:50:41.957000+00:00
584,13911,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EYLF65474,2022-07-21,2023-07-04T06:50:41.957000+00:00
585,13912,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ETCIX5105,2022-07-21,2023-07-04T06:50:41.957000+00:00
586,13913,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-07-04T06:50:41.957000+00:00
587,13914,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EU6RQ5270,2022-07-21,2023-07-04T06:50:41.957000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [30]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2612,46172,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-07-04T06:45:00.966000+00:00
2616,46179,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-07-04T06:45:00.966000+00:00
2617,46180,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-07-04T06:45:00.966000+00:00
2618,46181,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-07-04T06:45:00.966000+00:00
2619,46182,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-07-04T06:45:00.966000+00:00
2620,46183,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-07-04T06:45:00.966000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [31]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
37,4993,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2023-07-05,2023-07-05T02:53:39.395000+00:00
38,4994,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2023-07-05,2023-07-05T02:53:39.395000+00:00
39,4995,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2023-07-05,2023-07-05T02:53:39.395000+00:00
40,4996,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2023-07-05,2023-07-05T02:53:39.395000+00:00


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [32]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
38266,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [33]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
5777,FRSDGEMBDG512,FRSDGPMBDG512,"[2.451322, 45.66523]",2
5778,FRSDGEMBDG511,FRSDGPMBDG511,"[2.451322, 45.66523]",2


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [34]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
13606,FRIENE35353A11,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13607,FRIENE35353A51,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13608,FRIENE35353A41,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13609,FRIENE35353A31,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
13610,FRIENE35353A21,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [35]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PBARREOUCHEMOMORT', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
39644,40014,pascal.lhermitte@siege27.fr,SIEGE265,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTD,2021-06-15,2023-07-04T06:48:53.470000+00:00
39673,40043,pascal.lhermitte@siege27.fr,SIEGE266,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTG,2021-06-15,2023-07-04T06:48:53.470000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [36]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
39794,40176,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2023-07-04T06:48:44.762000+00:00
39808,40190,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2023-07-04T06:48:44.762000+00:00


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues

In [37]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1902', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
11470,11538,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19027,2023-06-02,2023-07-04T06:51:38.049000+00:00
11484,11552,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19026,2023-06-02,2023-07-04T06:51:38.049000+00:00
11486,11554,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19025,2023-06-02,2023-07-04T06:51:38.049000+00:00
11487,11555,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19024,2023-06-02,2023-07-04T06:51:38.049000+00:00
11488,11556,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19023,2023-06-02,2023-07-04T06:51:38.049000+00:00
11489,11557,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19022,2023-06-02,2023-07-04T06:51:38.049000+00:00
11490,11558,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19021,2023-06-02,2023-07-04T06:51:38.049000+00:00


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [38]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
800,26302,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540011,2023-03-17,2023-07-04T06:49:28.447000+00:00
809,26856,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540013,2023-03-17,2023-07-04T06:49:28.447000+00:00
810,26860,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540012,2023-03-17,2023-07-04T06:49:28.447000+00:00
850,28499,support@alizecharge.fr,pass pass électrique,"[2.460441, 50.78763]",Pole Gare 59670 BAVINCHOVE,FRH14P59054001,BAVINCHOVE - Pole Gare,Voirie,4,Accès libre,Mo-Su 00:00-23:59,FALSE,FRH14E590540014,2023-03-17,2023-07-04T06:49:28.447000+00:00
1286,40258,support@passpasselectrique.fr,Pass Pass Electrique,"[2.460441, 50.78763]",1bis Rue De La Gare 59670 Bavinchove,FRH14E59054001,Bavinchove - Gare,Parking public,2,Accès libre,24/7,false,FRH14E59054001,2023-01-27,2023-07-04T06:48:34.946000+00:00


### données d'un opérateur
- exemple Virta

In [39]:
itinerance_operateur = itinerance_5_full.loc[itinerance_5_full.contact_operateur == 'support@virta.global']
#itinerance_operateur.loc[itinerance_operateur['adresse_station - coordonneesXY'] == False, relations]
itinerance_operateur.loc[itinerance_operateur['nom_station - id_station_itinerance'] == False, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1370,42767,support@virta.global,MODULO,"[1.143419,47.25306]",Chateau de Montpoupon,FRS37P37046C,CERE LA RONDE 1372 - Château,Voirie,2,Accès libre,24/7,False,FRS37E1372,2023-05-31,2023-07-04T06:46:52.168000+00:00
1371,42768,support@virta.global,MODULO,"[1.143419,47.25306]",Chateau de Montpoupon,FRS37P37046C,CERE LA RONDE 1371 - Château,Voirie,2,Accès libre,24/7,False,FRS37E1371,2023-05-31,2023-07-04T06:46:52.168000+00:00
1372,42769,support@virta.global,MODULO,"[0.393657,47.00000]",Rue Principale,FRS37P37034,Braslou 11017 - Rue Principale,Voirie,2,Accès libre,24/7,False,FRS37E11017,2023-05-31,2023-07-04T06:46:52.168000+00:00
1373,42770,support@virta.global,MODULO,"[0.393657,47.00000]",Rue principale,FRS37P37034,Braslou 11018 - Rue Principale,Voirie,2,Accès libre,24/7,False,FRS37E11018,2023-05-31,2023-07-04T06:46:52.168000+00:00
1374,42771,support@virta.global,MODULO,"[1.081558,47.02746]",Rue du chemin vert,FRS37P37039,BRIDORE 1467,Voirie,2,Accès libre,24/7,False,FRS37E1467,2023-05-31,2023-07-04T06:46:52.168000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2493,44129,support@virta.global,MODULO,"[1.329051,48.07064]",Rue Ernest Gaugiran,FRS41P41106,LAMOTTE-BEUVRON- 117722- Rue Ernest Gaugiran,Voirie,2,Accès libre,24/7,False,FRS41E117722,2023-05-31,2023-07-04T06:46:52.168000+00:00
2494,44130,support@virta.global,MODULO,"[1.457725,47.56282]",Rue de l'hotel de ville,FRS41P41149,MONT PRES CHAMBORD 4435 - Rue de l'hotel de ville,Voirie,2,Accès libre,24/7,False,FRS41E4435,2023-05-31,2023-07-04T06:46:52.168000+00:00
2495,44131,support@virta.global,MODULO,"[1.457725,47.56282]",Rue de l'hotel de ville,FRS41P41149,MONT PRES CHAMBORD 4434 - Rue de l'hotel de ville,Voirie,2,Accès libre,24/7,False,FRS41E4434,2023-05-31,2023-07-04T06:46:52.168000+00:00
2496,44132,support@virta.global,MODULO,"[1.8285015,47.28266]",Route de Mennetou-Sur-Cher,FRS41P41110,LANGON 5162 - Route de Mennetou-Sur-Cher,Voirie,2,Accès libre,24/7,False,FRS41E5162,2023-05-31,2023-07-04T06:46:52.168000+00:00
